In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ReduceLROnPlateau, EarlyStopping,
                                        ModelCheckpoint)
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

# 1. Data Loading and Preprocessing
data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()
emotions = data['emotion'].values

# Convert pixel strings to numpy arrays
X = np.array([np.fromstring(pixel_sequence, dtype='float32', sep=' ')
              for pixel_sequence in pixels])

# Normalize pixel values
X /= 255.0

# Reshape to (num_samples, 48, 48, 1)
X = X.reshape(-1, 48, 48, 1)

# Convert labels to categorical
y = to_categorical(emotions, num_classes=7)

# Shuffle data
X, y = shuffle(X, y, random_state=42)

# Split into training and testing sets
num_samples = X.shape[0]
num_train_samples = int(0.8 * num_samples)
X_train = X[:num_train_samples]
Y_train = y[:num_train_samples]
X_test = X[num_train_samples:]
Y_test = y[num_train_samples:]

# 2. Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# 3. Define the Simple CNN Model
def create_simple_cnn(input_shape=(48, 48, 1), num_classes=7):
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_simple_cnn()

# 4. Configure Callbacks
callbacks = [
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        verbose=1,
        min_delta=1e-4
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        verbose=1,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        'best_simple_cnn_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# 5. Train the Model (without class_weight)
batch_size = 64
epochs = 100

history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=batch_size, shuffle=True),
    validation_data=(X_test, Y_test),
    epochs=epochs,
    callbacks=callbacks,
    verbose=2
)

# 6. Evaluate the Model
scores = model.evaluate(X_test, Y_test, verbose=0)
print(f'Test Accuracy: {scores[1]*100:.2f}%')

# 7. Generate Classification Report and Confusion Matrix
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

print('Classification Report')
print(classification_report(y_true, y_pred, digits=4))

print('Confusion Matrix')
cm = confusion_matrix(y_true, y_pred)

# 8. Visualize the Confusion Matrix
# (Code for plotting the confusion matrix)

# 9. Plot Accuracy and Loss
# (Code for plotting the graphs)

# 10. Save the Model
model.save('emotion_detection_simple_cnn.keras')


Epoch 1/100


c:\Users\Oana\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Oana\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.30301, saving model to best_simple_cnn_model.keras
449/449 - 47s - 105ms/step - accuracy: 0.2138 - loss: 2.5804 - val_accuracy: 0.3030 - val_loss: 1.8372 - learning_rate: 1.0000e-04
Epoch 2/100

Epoch 2: val_accuracy improved from 0.30301 to 0.40513, saving model to best_simple_cnn_model.keras
449/449 - 47s - 105ms/step - accuracy: 0.2771 - loss: 2.1962 - val_accuracy: 0.4051 - val_loss: 1.6523 - learning_rate: 1.0000e-04
Epoch 3/100

Epoch 3: val_accuracy improved from 0.40513 to 0.40917, saving model to best_simple_cnn_model.keras
449/449 - 51s - 114ms/step - accuracy: 0.3101 - loss: 2.0213 - val_accuracy: 0.4092 - val_loss: 1.6046 - learning_rate: 1.0000e-04
Epoch 4/100

Epoch 4: val_accuracy improved from 0.40917 to 0.43982, saving model to best_simple_cnn_model.keras
449/449 - 50s - 112ms/step - accuracy: 0.3425 - loss: 1.8952 - val_accuracy: 0.4398 - val_loss: 1.4965 - learning_rate: 1.0000e-04
Epoch 5/100

Epoch 5: val_accuracy impr